In [1]:
# imports for part 1

import os
import numpy
import pandas as pd
from sqlalchemy import create_engine
import sqlite3

In [2]:
file_name = "Best-Movies-Netflix.csv"
file_name2 = "Best-Shows-Netflix.csv"
                                                
data_dir = os.path.join(os.getcwd())                           
data_file = os.path.join(data_dir, file_name)
data_file2 = os.path.join(data_dir, file_name2) 
df = pd.read_csv(data_file, header=0, index_col=0)                
df = pd.DataFrame(df)
df2 = pd.read_csv(data_file2, header=0, index_col=0)                
df2 = pd.DataFrame(df2)

df.head()
df2.head()

,TITLE,RELEASE_YEAR,SCORE,NUMBER_OF_VOTES,DURATION,NUMBER_OF_SEASONS,MAIN_GENRE,MAIN_PRODUCTION
index,,,,,,,,
0,Breaking Bad,2008,9.5,1727694,48,5,drama,US
1,Avatar: The Last Airbender,2005,9.3,297336,24,3,scifi,US
2,Our Planet,2019,9.3,41386,50,1,documentary,GB
3,Kota Factory,2019,9.3,66985,42,2,drama,IN
4,The Last Dance,2020,9.1,108321,50,1,documentary,US


In [3]:
# combine both the movies and shows dataframes 

# need to add a column to movies for seasons
df['NUMBER_OF_SEASONS'] = 0

#merge the two dataframes together
frames = [df, df2]
newdf = pd.concat(frames)

newdf.tail() #head shows movies with seasons = 0 and tv shows have 1 or more

,TITLE,RELEASE_YEAR,SCORE,NUMBER_OF_VOTES,DURATION,MAIN_GENRE,MAIN_PRODUCTION,NUMBER_OF_SEASONS
index,,,,,,,,
241,Evil Genius,2018,7.5,27516,48,crime,US,1
242,13 Reasons Why,2017,7.5,282373,58,drama,US,4
243,Lupin,2021,7.5,100575,46,crime,FR,3
244,All of Us Are Dead,2022,7.5,41393,61,action,KR,1
245,I Am Not Okay with This,2020,7.5,56459,21,comedy,US,1


In [4]:
# Part 2
import pymongo
from pymongo import MongoClient

In [5]:
# Converting dataframe to dictionary
data = newdf.to_dict(orient = 'records')
# Making a Connection with MongoClient
client = MongoClient("mongodb://localhost:27017/")
# database
db = client["DS2002final"]
# collection
db.netflix.insert_many(data)

In [6]:
# chatbox import statements
import nltk
from nltk import word_tokenize,sent_tokenize
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
nltk.download('punkt')
#read more on the steamer https://towardsdatascience.com/stemming-lemmatization-what-ba782b7c0bd8
import numpy as np 
import tflearn
import tensorflow as tf
import random
import json
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amsha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Scipy not supported!


In [7]:
# Read in intents.json file
with open("intents.json") as file:
    data = json.load(file)


try:
    with open("data.pickle","rb") as f:
        words, labels, training, output = pickle.load(f)

except:
    words = []
    labels = []
    docs_x = []
    docs_y = []
    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])
            
        if intent["tag"] not in labels:
            labels.append(intent["tag"])


    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))
    labels = sorted(labels)

    training = []
    output = []
    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
               bag.append(1)
            else:
              bag.append(0)
    
        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1
        
        training.append(bag)
        output.append(output_row)

    training = np.array(training)
    output = np.array(output)
    
    with open("data.pickle","wb") as f:
        pickle.dump((words, labels, training, output), f)

In [8]:
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

try:
    model.load("model.tflearn")
except:
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")

Training Step: 6999  | total loss: 0.00984 | time: 0.029s
| Adam | epoch: 1000 | loss: 0.00984 - acc: 0.9999 -- iter: 48/56
Training Step: 7000  | total loss: 1.48351 | time: 0.033s
| Adam | epoch: 1000 | loss: 1.48351 - acc: 0.9124 -- iter: 56/56
--
INFO:tensorflow:C:\Users\amsha\Downloads\DS2002Final\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:Restoring parameters from C:\Users\amsha\Downloads\DS2002Final\model.tflearn


In [9]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    
    return np.array(bag)

In [ ]:
def chat():
    print("Start talking with the bot! (type quit to stop)")
    print("You can ask the bot:")
    print("1. Top movies for each year?")
    print("2. top tv shows for each year?")
    print("3. top movies per decade?") 
    print("4. top movie by genre?")
    print("5. oldest movie from france?")
    print("6. longest movie and tv show with the most seasons?")
    print("7. top movie by number of votes?")  
    print("8. what is the longest duration for a tv episode?")
    print("9. what's the worst romance movie?")
    print("10. top tv show by genre?")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        result = model.predict([bag_of_words(inp, words)])[0]
        
        result_index = np.argmax(result)
       
        tag = labels[result_index]
       

        if result[result_index] >= 0.7:
            
        
            for tg in data["intents"]:
 
                if tg['tag'] == tag and tag == 'firstq':
                   
                    print('You asked question 1!')
                    #responses = tg['responses']
                    # 1) What are the top movies for each year?
                    # section off by year and then find highest score for given year 
                    i = 1954
                    while i <= 2022:
                        
                        try:
                            year = db.netflix.find({"RELEASE_YEAR":i,'NUMBER_OF_SEASONS':0}).sort("SCORE", -1)[0]
                       
                            print(year['TITLE'], year["RELEASE_YEAR"])
                            i = i + 1
                        except:
                            i = i+1
                    
                
                
                elif tg['tag'] == tag and tag == 'secondq':
                    print("You asked question 2!")
                    # 2) Top tv shows for each year
                    i = 1954
                    while i <= 2022:
                        
                        try:
                            year = db.netflix.find({"RELEASE_YEAR":i, "NUMBER_OF_SEASONS":{ '$gt' : 0 }}).sort("SCORE", -1)[0]
                       
                            print(year['TITLE'], year["RELEASE_YEAR"])
                            i = i + 1
                        except:
                            i = i+1
                    
                    print()
                
                elif tg['tag'] == tag and tag == 'thirdq':
                    print('You asked question 3!')
                    #3) Top movies per decade
                    
                    i = 1950
                    while i <= 2022:
                        
                        try:
                            year = db.netflix.find({"RELEASE_YEAR":{'$gte' :i, '$lt' :i+10 }, "NUMBER_OF_SEASONS":0}).sort("SCORE", -1)[0]
#                           print(type(year['RELEASE_YEAR'])) 
                            decade = str(year['RELEASE_YEAR'])
                            decade = decade[:3] + '0s'
                            print(year['TITLE'], decade)
                            i = i + 10
                        except:
                            i = i + 10
                    print()
                
                elif tg['tag'] == tag and tag == 'fourthq':
                    print('You asked question 4!')
                    #4) Top movies by genre
                    ans = input("Select one of the following options: drama,animation, western, war, thriller, sports, romance, musical, horror, fantasy, documentary, comedy, scifi, action, crime, all")
                    
                    drama = db.netflix.find({'NUMBER_OF_SEASONS': 0,'MAIN_GENRE':'drama'}).sort("SCORE", -1)[0]
                    animation= db.netflix.find({'NUMBER_OF_SEASONS': 0,'MAIN_GENRE':'animation'}).sort("SCORE", -1)[0]
                    western= db.netflix.find({'NUMBER_OF_SEASONS': 0,'MAIN_GENRE':'western'}).sort("SCORE", -1)[0]
                    war= db.netflix.find({'NUMBER_OF_SEASONS': 0,'MAIN_GENRE':'war'}).sort("SCORE", -1)[0]
                    thriller= db.netflix.find({'NUMBER_OF_SEASONS': 0,'MAIN_GENRE':'thriller'}).sort("SCORE", -1)[0]
                    sports= db.netflix.find({'NUMBER_OF_SEASONS': 0,'MAIN_GENRE':'sports'}).sort("SCORE", -1)[0]
                    romance = db.netflix.find({'NUMBER_OF_SEASONS': 0,'MAIN_GENRE':'romance'}).sort("SCORE", -1)[0]
                    musical= db.netflix.find({'NUMBER_OF_SEASONS': 0,'MAIN_GENRE':'musical'}).sort("SCORE", -1)[0]
                    fantasy= db.netflix.find({'NUMBER_OF_SEASONS': 0,'MAIN_GENRE':'fantasy'}).sort("SCORE", -1)[0]
                    documentary= db.netflix.find({'NUMBER_OF_SEASONS': 0,'MAIN_GENRE':'documentary'}).sort("SCORE", -1)[0]
                    comedy = db.netflix.find({'NUMBER_OF_SEASONS': 0,'MAIN_GENRE':'comedy'}).sort("SCORE", -1)[0]
                    scifi = db.netflix.find({'NUMBER_OF_SEASONS': 0,'MAIN_GENRE':'scifi'}).sort("SCORE", -1)[0]
                    action = db.netflix.find({'NUMBER_OF_SEASONS': 0,'MAIN_GENRE':'action'}).sort("SCORE", -1)[0]
                    crime = db.netflix.find({'NUMBER_OF_SEASONS': 0,'MAIN_GENRE':'crime'}).sort("SCORE", -1)[0]
                       
                    if ans.lower() == 'drama':
                        print(drama["TITLE"], "(",drama['MAIN_GENRE'],")")
                    elif ans.lower() == 'animation':
                        print(animation["TITLE"], "(",animation['MAIN_GENRE'],")")
                    elif ans.lower() == 'western':
                        print(western["TITLE"], "(",western['MAIN_GENRE'],")")
                    elif ans.lower() == 'war':
                        print(war["TITLE"], "(",war['MAIN_GENRE'],")")
                    elif ans.lower() == 'thriller':
                        print(thriller["TITLE"], "(",thriller['MAIN_GENRE'],")")
                    elif ans.lower() == 'sports':
                        print(sports["TITLE"], "(",sports['MAIN_GENRE'],")")
                    elif ans.lower() == 'romance':
                        print(romance["TITLE"], "(",romance['MAIN_GENRE'],")")
                    elif ans.lower() == 'musical':
                        print(musical["TITLE"], "(",musical['MAIN_GENRE'],")")
                    elif ans.lower() == 'fantasy':
                        print(fantasy["TITLE"], "(",fantasy['MAIN_GENRE'],")")
                    elif ans.lower() == 'documentary':
                        print(documentary["TITLE"], "(",documentary['MAIN_GENRE'],")")
                    elif ans.lower() == 'comedy':
                        print(comedy["TITLE"], "(",comedy['MAIN_GENRE'],")")
                    elif ans.lower() == 'scifi':
                        print(scifi["TITLE"], "(",scifi['MAIN_GENRE'],")")
                    elif ans.lower() == 'action':
                        print(action["TITLE"], "(",action['MAIN_GENRE'],")")
                    elif ans.lower() == 'crime':
                        print(crime['TITLE'], "(",crime['MAIN_GENRE'],")")
                    elif ans.lower() == 'all':
                        print(drama["TITLE"], "(",drama['MAIN_GENRE'],")")
                        print(western["TITLE"], "(",western['MAIN_GENRE'],")")
                        print(war["TITLE"], "(",war['MAIN_GENRE'],")")
                        print(thriller["TITLE"], "(",thriller['MAIN_GENRE'],")")
                        print(sports["TITLE"], "(",sports['MAIN_GENRE'],")")
                        print(scifi["TITLE"], "(",scifi['MAIN_GENRE'],")")
                        print(romance["TITLE"], "(",romance['MAIN_GENRE'],")")
                    
                        print(musical["TITLE"], "(",musical['MAIN_GENRE'],")")
                   
                        print(horror["TITLE"], "(",horror['MAIN_GENRE'],")")
                    
                        print(fantasy['TITLE'], "(",fantasy['MAIN_GENRE'],")")
                        print(documentary["TITLE"], "(",documentary['MAIN_GENRE'],")")
                        print(crime["TITLE"], "(",crime['MAIN_GENRE'],")")
                        print(comedy["TITLE"], "(",comedy['MAIN_GENRE'],")")
                        print(animation["TITLE"], "(",animation['MAIN_GENRE'],")")
                        print(action["TITLE"], "(",action['MAIN_GENRE'],")")
                        
                    else:
                        print("Please choose question 4 again and choose a valid genre (drama,animation, western, war, thriller, sports, romance, musical, horror, fantasy, documentary, comedy, scifi, action, crime, all)")
                           
                    print()   
                        
                        
                elif tg['tag'] == tag and tag== 'fifthq':
                    print('You picked question 5!')
                    
                    #5) Oldest movie and tv show from France?
                    france_movie = db.netflix.find({'NUMBER_OF_SEASONS': 0, "MAIN_PRODUCTION": "FR"}).sort("RELEASED_YEAR", 1)[0]
                   
                    france_show = db.netflix.find({'NUMBER_OF_SEASONS': {'$gt':0}}).sort("RELEASED_YEAR", 1)[0]
                    print("Oldest Movie from France:", france_movie['TITLE'])
                    print("Oldest TV Show from France:", france_show['TITLE'])
                    print()
#                   print("Movie:",production_company_movie['TITLE'], "TV Show:", productoin_company_show['TITLE'])
                    
#                     print()
                elif tg['tag'] == tag and tag =='sixthq':
                    print('You picked question 6!')
                    # 7) longest movie and tv show with the most seasons?
                    longest_movie = db.netflix.find({'NUMBER_OF_SEASONS' : 0}).sort("DURATION", -1)[0]
                    longest_show = db.netflix.find({'NUMBER_OF_SEASONS' : {'$gt':0}}).sort("NUMBER_OF_SEASONS", -1)[0]
                    print('Longest movie:', longest_movie['TITLE'],'has a duration of', longest_movie['DURATION'], 'minutes')
                    print("Longest show:", longest_show['TITLE'],'has',longest_show["NUMBER_OF_SEASONS"], 'seasons')
                    print()
                elif tg['tag'] == tag and tag == 'seventhq':
                    print('You picked question 7!')
                    # 7) Topmby number of votes?
                    top_ten_votes = db.netflix.find({'NUMBER_OF_SEASONS' : 0}).sort("NUMBER_OF_VOTES", -1)[0]
                    print(top_ten_votes['TITLE'], ":", top_ten_votes['NUMBER_OF_VOTES'], 'votes')
                    print()
               
                elif tg['tag'] == tag and tag =='eighthq':
                    print('You picked question 8!')
                    # 8)What is the longest duration of an epiosde?
                    duration = db.netflix.find({'NUMBER_OF_SEASONS' : {'$gt':0}}).sort("DURATION", -1)[0]
                    print(duration["TITLE"],'has a longest episode of',duration["DURATION"],'minutes.' )
                    print()
                elif tg['tag'] == tag and tag =='ninethq':
                    print('You picked question 9!')
                    # 9) What's the lowest rated romance movie?
                    lowest_movie = db.netflix.find({'NUMBER_OF_SEASONS' : 0, "MAIN_GENRE":'romance'}).sort("SCORE", 1)[0]
                    
                    print("Lowest rated romance movie:", lowest_movie['TITLE'],'has a score of',lowest_movie['SCORE'])
                    print()
                    
                elif tg['tag'] == tag and tag =='tenthq':
                    print('You picked question 10!')
                    #10) Top tv show for each genre?
                    ans = input("Select one of the following options: drama,animation,reality, western, war, thriller, romance, documentary, comedy, scifi, action, crime, all")
                    
                    drama = db.netflix.find({'NUMBER_OF_SEASONS': {'$gt': 0},'MAIN_GENRE':'drama'}).sort("SCORE", -1)[0]
                    animation= db.netflix.find({'NUMBER_OF_SEASONS': {'$gt': 0},'MAIN_GENRE':'animation'}).sort("SCORE", -1)[0]
                    western= db.netflix.find({'NUMBER_OF_SEASONS': {'$gt': 0},'MAIN_GENRE':'western'}).sort("SCORE", -1)[0]
                    war= db.netflix.find({'NUMBER_OF_SEASONS': {'$gt': 0},'MAIN_GENRE':'war'}).sort("SCORE", -1)[0]
                    thriller= db.netflix.find({'NUMBER_OF_SEASONS': {'$gt': 0},'MAIN_GENRE':'thriller'}).sort("SCORE", -1)[0]
                  
                    romance = db.netflix.find({'NUMBER_OF_SEASONS': {'$gt': 0},'MAIN_GENRE':'romance'}).sort("SCORE", -1)[0]
                  
                    reality= db.netflix.find({'NUMBER_OF_SEASONS': {'$gt': 0},'MAIN_GENRE':'reality'}).sort("SCORE", -1)[0]
                    
                    documentary= db.netflix.find({'NUMBER_OF_SEASONS': {'$gt': 0},'MAIN_GENRE':'documentary'}).sort("SCORE", -1)[0]
                    comedy = db.netflix.find({'NUMBER_OF_SEASONS': {'$gt': 0},'MAIN_GENRE':'comedy'}).sort("SCORE", -1)[0]
                    scifi = db.netflix.find({'NUMBER_OF_SEASONS': {'$gt': 0},'MAIN_GENRE':'scifi'}).sort("SCORE", -1)[0]
                    action = db.netflix.find({'NUMBER_OF_SEASONS': {'$gt': 0},'MAIN_GENRE':'action'}).sort("SCORE", -1)[0]
                    crime = db.netflix.find({'NUMBER_OF_SEASONS': {'$gt': 0},'MAIN_GENRE':'crime'}).sort("SCORE", -1)[0]
                       
                    if ans.lower() == 'drama':
                        print(drama["TITLE"], "(",drama['MAIN_GENRE'],")")
                    elif ans.lower() == 'animation':
                        print(animation["TITLE"], "(",animation['MAIN_GENRE'],")")
                    elif ans.lower() == 'western':
                        print(western["TITLE"], "(",western['MAIN_GENRE'],")")
                    elif ans.lower() == 'war':
                        print(war["TITLE"], "(",war['MAIN_GENRE'],")")
                    elif ans.lower() == 'thriller':
                        print(thriller["TITLE"], "(",thriller['MAIN_GENRE'],")")
                    elif ans.lower() == 'sports':
                        print(sports["TITLE"], "(",sports['MAIN_GENRE'],")")
                    elif ans.lower() == 'romance':
                        print(romance["TITLE"], "(",romance['MAIN_GENRE'],")")
                    elif ans.lower() == 'musical':
                        print(musical["TITLE"], "(",musical['MAIN_GENRE'],")")
                    elif ans.lower() == 'reality':
                        print(reality["TITLE"], "(",reality['MAIN_GENRE'],")")
                    elif ans.lower() == 'documentary':
                        print(documentary["TITLE"], "(",documentary['MAIN_GENRE'],")")
                    elif ans.lower() == 'comedy':
                        print(comedy["TITLE"], "(",comedy['MAIN_GENRE'],")")
                    elif ans.lower() == 'scifi':
                        print(scifi["TITLE"], "(",scifi['MAIN_GENRE'],")")
                    elif ans.lower() == 'action':
                        print(action["TITLE"], "(",action['MAIN_GENRE'],")")
                    elif ans.lower() == 'crime':
                        print(crime['TITLE'], "(",crime['MAIN_GENRE'],")")
                    elif ans.lower() == 'all':
                        print(drama["TITLE"], "(",drama['MAIN_GENRE'],")")
                        print(western["TITLE"], "(",western['MAIN_GENRE'],")")
                        print(war["TITLE"], "(",war['MAIN_GENRE'],")")
                        print(thriller["TITLE"], "(",thriller['MAIN_GENRE'],")")
                  
                        print(scifi["TITLE"], "(",scifi['MAIN_GENRE'],")")
                        print(romance["TITLE"], "(",romance['MAIN_GENRE'],")")
                        print(reality['TITLE'], "(",reality['MAIN_GENRE'],")")
                        print(documentary["TITLE"], "(",documentary['MAIN_GENRE'],")")
                        print(crime["TITLE"], "(",crime['MAIN_GENRE'],")")
                        print(comedy["TITLE"], "(",comedy['MAIN_GENRE'],")")
                        print(animation["TITLE"], "(",animation['MAIN_GENRE'],")")
                        print(action["TITLE"], "(",action['MAIN_GENRE'],")")
                        
                    else:
                        print("Please choose question 10 again and enter a valid genre (drama,animation,reality, western, war, thriller, romance, documentary, comedy, scifi, action, crime, all)")
                   
                    print() 
                    
                elif tg['tag'] == tag and tag == 'help':
                    print('You can choose any of the following!')
                    responses = tg['responses']
                    print(responses)
        else:
            print("The bot didn't quite understand that. You can try asking the following: ")
                
            print("1. Top movies for each year?")
            print("2. top tv shows for each year?")
            print("3. top movies per decade?") 
            print("4. top movie by genre?")
            print("5. oldest movie from france?")
            print("6. longest movie and tv show with the most seasons?")
            print("7. top movie by number of votes?")  
            print("8. what is the longest duration for a tv episode?")
            print("9. what's the worst romance movie?")
            print("10. top tv show by genre?")
chat()

Start talking with the bot! (type quit to stop)
You can ask the bot:
1. Top movies for each year?
2. top tv shows for each year?
3. top movies per decade?
4. top movie by genre?
5. oldest movie from france?
6. longest movie and tv show with the most seasons?
7. top movie by number of votes?
8. what is the longest duration for a tv episode?
9. what's the worst romance movie?
10. top tv show by genre?
You: help
You can choose any of the following!
["1. Top movies for each year? , 2. top tv shows for each year? , 3. top movies per decade? , 4. top movie by genre? , 5. oldest movie from france? , 6. longest movie and tv show with the most seasons? , 7. top movie by number of votes? , 8. what is the longest duration for a tv episode? , 9. what's the worst romance movie? , 10. top tv show by genre?"]
